# **Projeto IA - Connect 4**

### Membros do Grupo:
- Maximiliano Vítor Phillips e Sá (up202305979)
- Orlando Miguel Carvalho-Soares (up202303606)
- Rui Jorge Pereira Rua (up202305259)

## Índice
1. Introdução
2. Código Universal
3. Game Modes
    - Humano x Humano
    - Humano x Computador
    - Computador x Computador


## 1. Introdução

Este projeto tem como objetivo desenvolver e implementar um programa capaz de jogar 'Connect-4' com um humano. O programa tem 4 modos: Humano-Humano, Humano-Computador(MCTS), Humano-Computador(Árvores de Decisão) e Computador-Computador. O programa utiliza os algoritmos MCTS e Árvores de Decisão usando o procedimento ID3 no modo Computador-Computador.


## 2. Código Universal

O seguinte código é 'universal' e serve para gerir o estado atual do jogo.

meta.py serve para guardar as definições básicas do jogo e o valor C para o algoritmo MCTS.

ConnectState.py guarda o estado do jogo e as funções básicas para UI output e para verificar se existe jogadas legais e se existe uma vitória. 

meta.py

In [ ]:
import math

#meta dados relativos ao jogo
class GameMeta:
    PLAYERS = {'none': 0, 'one': 1, 'two': 2}
    OUTCOMES = {'none': 0, 'one': 1, 'two': 2, 'draw': 3}
    INF = float('inf')
    ROWS = 6
    COLS = 7
    GAMEMODE = None

#valor C do UCB
class MCTSMeta:
    C = math.sqrt(2)



ConnectState.py

In [ ]:
from copy import deepcopy
import numpy as np
from meta import GameMeta


class ConnectState:
    '''
        Construtor do board e de alguns metadados sobre o estado atual. 
        Ex: jogador atual, jogada anterior, altura de cada coluna (height[col] = 0 -> coluna col cheia)
    '''

    def __init__(self):
        self.board = [[0] * GameMeta.COLS for _ in range(GameMeta.ROWS)]
        self.to_play = GameMeta.PLAYERS['one']
        self.height = [GameMeta.ROWS - 1] * GameMeta.COLS
        self.last_played = []

    #retorna uma cópia do estado atual
    def get_board(self):
        return deepcopy(self.board)
    
    #dá print_board ao board atual na consola
    def print_board(self):
        print('=============================')

        for row in range(GameMeta.ROWS):
            for col in range(GameMeta.COLS):
                print('| {} '.format('X' if self.board[row][col] == 1 else 'O' if self.board[row][col] == 2 else ' '), end='')
            print('|')

        print('=============================')
        print('  1   2   3   4   5   6   7')

    #marca a jogada escolhida pelo jogador no board, e dá update a alguns metadados do estado atual
    def move(self, col):
        self.board[self.height[col]][col] = self.to_play
        self.last_played = [self.height[col], col]
        self.height[col] -= 1
        self.to_play = GameMeta.PLAYERS['two'] if self.to_play == GameMeta.PLAYERS['one'] else GameMeta.PLAYERS['one']

    #retorna uma lista com as jogadas que ainda não estão cheias (height[col] != 0)
    def get_legal_moves(self):
        return [col for col in range(GameMeta.COLS) if self.board[0][col] == 0]

    #verfica se há vencedor e retorna o seu número (X = 1 ; O = 2), se não houver retorna 0
    def check_win(self):
        if len(self.last_played) > 0 and self.check_win_from(self.last_played[0], self.last_played[1]):
            return self.board[self.last_played[0]][self.last_played[1]]
        return 0

    def check_win_from(self, row, col):
        player = self.board[row][col]
        """
        Ultima jogada realizada está na posição (row, col)
        Verificar a grid 6x7 à volta à procura de um vencedor
        """

        consecutive = 1
        # verificar horizontal
        tmprow = row
        while tmprow + 1 < GameMeta.ROWS and self.board[tmprow + 1][col] == player:
            consecutive += 1
            tmprow += 1
        tmprow = row
        while tmprow - 1 >= 0 and self.board[tmprow - 1][col] == player:
            consecutive += 1
            tmprow -= 1

        if consecutive >= 4:
            return True

        # verificar vertical
        consecutive = 1
        tmpcol = col
        while tmpcol + 1 < GameMeta.COLS and self.board[row][tmpcol + 1] == player:
            consecutive += 1
            tmpcol += 1
        tmpcol = col
        while tmpcol - 1 >= 0 and self.board[row][tmpcol - 1] == player:
            consecutive += 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # verificar diagonal1
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol + 1 < GameMeta.COLS and self.board[tmprow + 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol += 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol - 1 >= 0 and self.board[tmprow - 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol -= 1

        if consecutive >= 4:
            return True

        # verificar diagonal2
        consecutive = 1
        tmprow = row
        tmpcol = col
        while tmprow + 1 < GameMeta.ROWS and tmpcol - 1 >= 0 and self.board[tmprow + 1][tmpcol - 1] == player:
            consecutive += 1
            tmprow += 1
            tmpcol -= 1
        tmprow = row
        tmpcol = col
        while tmprow - 1 >= 0 and tmpcol + 1 < GameMeta.COLS and self.board[tmprow - 1][tmpcol + 1] == player:
            consecutive += 1
            tmprow -= 1
            tmpcol += 1

        if consecutive >= 4:
            return True

        return False

    #verifica se o jogo já acabou verificando se há vencedor ou se ainda existem jogadas possíveis
    def game_over(self):
        return self.check_win() or len(self.get_legal_moves()) == 0

    #verifica quem ganhou ou se é empate
    def get_result(self):
        winner = self.check_win()
        if len(self.get_legal_moves()) == 0 and winner == 0:
            return GameMeta.OUTCOMES['draw']
        return GameMeta.OUTCOMES['one'] if winner == GameMeta.PLAYERS['one'] else GameMeta.OUTCOMES['two']



## 3. Algoritmos Usados

Foram usados dois algoritmos diferentes:
1. Monte Carlo Tree Search (MCTS)
2. Árvores de Decisão (Procedimento ID3)

### 1. Monte Carlo Tree Search (MCTS)

mcts.py

In [ ]:
import random
import time
import math
from copy import deepcopy

from ConnectState import ConnectState
from meta import GameMeta, MCTSMeta


class Node:
    '''
        construtor da classe Node que guarda valores como o numero de visitas ao Nó, 
        número de vitórias, os seus filhos, o seu pai
    '''
    def __init__(self, move, parent):
        self.move = move
        self.parent = parent
        self.N = 0 # num times visited (current node)
        self.W = 0 # num wins for current node
        self.children = {}
        self.outcome = GameMeta.PLAYERS['none']

    #adiciona os filhos ao dicionário do "children" do pai
    def add_children(self, children: dict) -> None:
        for child in children:
            self.children[child.move] = child

    #calcular UCB do nó
    def value(self, c: float = MCTSMeta.C):
        if self.N == 0:
            return 0 if c == 0 else GameMeta.INF
        else:
            return self.W / self.N + c * math.sqrt(math.log(self.parent.N) / self.N)


class MCTS:
    #Construtor da classe MCTS que guarda algumas estatísticas e é o ponto inicial do algoritmo
    def __init__(self, state=ConnectState()):
        self.root_state = deepcopy(state)
        self.root = Node(None, None)
        self.run_time = 0
        self.node_count = 0
        self.num_rollouts = 0
        self.best_trivial = None  # armazena jogada imediata/defensiva

    # verfica se o computador pode ganhar numa jogada só
    def check_instant_win(self, state: ConnectState) -> int:
        for move in state.get_legal_moves():
            st = deepcopy(state)
            st.move(move)
            if st.game_over():
                return move
        return -1
    
    def check_block_opponent(self, state: ConnectState) -> int:
        '''Se o oponente tem jogada de vitória imediata, retorna movimento para bloqueá-la.'''
        opponent = 3 - state.to_play
        for move in state.get_legal_moves():
            st = deepcopy(state)
            # força o próximo para oponente
            st.to_play = opponent
            st.move(move)
            if st.game_over():
                return move
        return -1
    
    # verfica se o computador só tem uma coluna para jogar
    def check_one_move_available(self, state: ConnectState) -> int:
        legalMoves = state.get_legal_moves()
        if len(legalMoves) == 1:
            return legalMoves[0]
        else:
            return -1

    #escolhe o próximo nó/estado a ser explorado
    def select_node(self) -> tuple:
        node = self.root
        state = deepcopy(self.root_state)

        #escolhe o nó filho com maior UCB (se o pai já tiver sido expandido)
        while len(node.children) != 0:
            children = node.children.values()
            max_value = max(children, key=lambda n: n.value()).value()
            max_nodes = [n for n in children if n.value() == max_value]

            node = random.choice(max_nodes)
            state.move(node.move)

            if node.N == 0:
                return node, state
            
        #escolhe aleatóriamente um nó filho para expandir
        if self.expand(node, state):
            node = random.choice(list(node.children.values()))
            state.move(node.move)

        return node, state

    #adiciona os estados seguintes como nós filho do estado atual
    def expand(self, parent: Node, state: ConnectState) -> bool:
        if state.game_over():
            return False

        children = [Node(move, parent) for move in state.get_legal_moves()]
        parent.add_children(children)

        return True

    #simula aleatóriamente um jogo e retorna o vencedor
    def simulate(self, state: ConnectState) -> int:
        while not state.game_over():
            state.move(random.choice(state.get_legal_moves()))

        return state.get_result()

    #faz backtrack aos nós escolhidos na fase de rollout e dá update aos seus valores
    def back_propagate(self, node: Node, turn: int, outcome: int) -> None:
        # Para o jogador atual
        reward = 0 if outcome == turn else 1

        while node is not None:
            node.N += 1
            node.W += reward
            node = node.parent
            if outcome == GameMeta.OUTCOMES['draw']:
                reward = 0
            else:
                reward = 1 - reward

    #procura a melhor jogada por "time_limit" segundos
    def search(self, time_limit: int):
        # 1) Vitória imediata
        win = self.check_instant_win(self.root_state)
        if win != -1:
            self.run_time = 0
            self.num_rollouts = 1
            self.best_trivial = win
            return
        # 2) Bloquear vitória do oponente
        block = self.check_block_opponent(self.root_state)
        if block != -1:
            self.run_time = 0
            self.num_rollouts = 1
            self.best_trivial = block
            return
        # 3) Única jogada possível
        one = self.check_one_move_available(self.root_state)
        if one != -1:
            self.run_time = 0
            self.num_rollouts = 1
            self.best_trivial = one
            return
        start_time = time.process_time()

        num_rollouts = 0
        while time.process_time() - start_time < time_limit:
            node, state = self.select_node()
            outcome = self.simulate(state)
            self.back_propagate(node, state.to_play, outcome)
            num_rollouts += 1

        self.run_time = time.process_time() - start_time
        self.num_rollouts = num_rollouts
        self.best_trivial = None

    #retorna a melhor jogada que o mcts encontrou
    def best_move(self):
        # Se root já concluída por heurística:
        if self.best_trivial is not None:
            return self.best_trivial
        
        if self.root_state.game_over():
            return -1
        
        safes = []
        for move, node in self.root.children.items():
            copy_state = deepcopy(self.root_state)
            copy_state.move(move)
            # se o oponente não consegue ganhar depois da nossa jogada
            if self.check_instant_win(copy_state) == -1:
                safes.append(node)
        '''
        # test print to check safes
        print("safes: ")
        s = []
        for i in range(len(safes)):
            s.append(safes[i].move)
        print(s)
        '''
        if safes:
            best_safe = max(safes, key=lambda n: n.value())
            return best_safe.move
        else:   
            max_value = max(self.root.children.values(), key=lambda n: n.N).N
            max_nodes = [n for n in self.root.children.values() if n.N == max_value]
            best_child = random.choice(max_nodes)
            return best_child.move

    #realiza (em definitivo) a melhor jogada que encontrou
    def move(self, move):
        if move in self.root.children:
            self.root_state.move(move)
            self.root = self.root.children[move]
            return

        self.root_state.move(move)
        self.root = Node(None, None)

    #retorna estatísticas
    def statistics(self) -> tuple:
        return self.num_rollouts, self.run_time
    
    # mudar o valor de c para 'atacar' mais em vez de focar em exploração
    def change_c_value(self):
        MCTSMeta.C = 1


## 4. Game Modes

Existem 4 modos de jogo:
1. Humano x Humano
2. Humano x Computador (MCTS)
3. Humano x Computador (Árvores de Decisão)
4. Computador x Computador (MCTS x Árvores de Decisão)

### Código Base para a Interface

game.py

In [ ]:
from ConnectState import ConnectState
from mcts import MCTS
from meta import GameMeta
from tree import trainTree
import numpy as np
import random

def playPvP():
    state = ConnectState()

    while not state.game_over():
        print("Current state:")
        state.print_board()

        # --- Player 1 Move ---
        while True:
            p1_input = input("Player 1 ('X') enter a move: ")
            if not p1_input.strip():
                print("You need to enter a move.")
                continue
            try:
                p1Move = int(p1_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue

            if p1Move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break

        state.move(p1Move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        # --- Player 2 Move ---
        while True:
            p2_input = input("Player 2 ('O') enter a move: ")
            if not p2_input.strip():
                print("You need to enter a move.")
                continue
            try:
                p2Move = int(p2_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue

            if p2Move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break

        state.move(p2Move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break


def playPvC1():
    state = ConnectState()
    mcts = MCTS(state)
    changed_To_Attack = False # se o valor de C foi mudado ou não
    num_Plays = 0

    while not state.game_over():
        print("Current state:")
        state.print_board()

        while True:
            user_input = input("Enter a move: ")
            # Check if input is blank or not a number
            if not user_input.strip():
                print("You need to enter a move.")
                continue
            try:
                user_move = int(user_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue
            if user_move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break  # valid input and legal move

        state.move(user_move)
        mcts.move(user_move)

        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        print("Thinking...")

        mcts.search(10)
        num_rollouts, run_time = mcts.statistics()
        print(f"Statistics: {num_rollouts} rollouts in {run_time:.2f} seconds")
        move = mcts.best_move()
        state.move(move)
        mcts.move(move)

        
        if state.game_over():
            state.print_board()
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break

        if not changed_To_Attack:
            num_Plays += 2
            if num_Plays > 17:
                mcts.change_c_value()
                changed_To_Attack = True

def playPvC2():
    state = ConnectState()
    tree = trainTree()

    while not state.game_over():
        print("Current state:")
        state.print_board()

        while True:
            user_input = input("Enter a move: ")
            # Check if input is blank or not a number
            if not user_input.strip():
                print("You need to enter a move.")
                continue
            try:
                user_move = int(user_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue
            if user_move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break  # valid input and legal move

        state.move(user_move)

        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        new_state = np.array(state.get_board()).flatten()

        predicted_move = tree.predict(np.array([new_state]))[0]

        legal_moves = state.get_legal_moves()

        if predicted_move not in legal_moves:
            state.move(legal_moves[random.randint(0, len(legal_moves)-1)])
        else:
            state.move(predicted_move)

        print("Tree chose to play in column: " + str(predicted_move+1))

        
        if state.game_over():
            state.print_board()
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break

def playCvC():
    state = ConnectState()
    tree = trainTree()
    mcts = MCTS(state)
    changed_To_Attack = False
    num_Plays = 0
    print("Current state:")
    state.print_board()

    while not state.game_over():
        # --- MCTS turn ---
        print("Thinking (MCTS)...")
        mcts.search(10)
        num_rollouts, run_time = mcts.statistics()
        print(f"Statistics: {num_rollouts} rollouts in {run_time:.2f} seconds")

        mcts_move = mcts.best_move()
        state.move(mcts_move)
        mcts.move(mcts_move)

        print("MCTS chose column:", mcts_move + 1)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("MCTS ('X') won!")
            else:
                print("Draw!")
            break

        # --- Tree turn ---
        new_state = np.array(state.get_board()).flatten()
        predicted_move = tree.predict(np.array([new_state]))[0]

        legal_moves = state.get_legal_moves()
        if predicted_move not in legal_moves:
            # fallback to random legal move
            predicted_move = random.choice(legal_moves)

        state.move(predicted_move)
        mcts.move(predicted_move)

        print("Tree chose column:", predicted_move + 1)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Tree ('O') won!")
            else:
                print("Draw!")
            break

        # Optionally adjust MCTS’s C parameter after a number of full turns
        if not changed_To_Attack:
            num_Plays += 2
            if num_Plays > 17:
                mcts.change_c_value()
                changed_To_Attack = True
    


'''
    Inicializador do modo de jogo pretendido:
    1. PvP
    2. PvC (mcts)
    3. PvC (tree)
    4. CvC
'''
if __name__ == "__main__":
    print("Welcome to Connect 4!\nPlease select one of the following game modes by selecting its corresponding number:\n1. Human vs. Human\n2. Human vs. Computer(mcts)\n3. Human vs. Computer(tree)\n4. Computer vs. Computer")
    GameMeta.GAMEMODE = int(input("Select option:"))
    while 1 > GameMeta.GAMEMODE or GameMeta.GAMEMODE > 4:
        print("Invalid Gamemode")
        GameMeta.GAMEMODE = int(input("Select option:"))
    if GameMeta.GAMEMODE == 1:
        playPvP()
    elif GameMeta.GAMEMODE == 2:
        playPvC1()
    elif GameMeta.GAMEMODE == 3:
        playPvC2()
    else:
        playCvC()


### 1. Humano x Humano

A função seguinte gere o modo PvP aceitando entradas de utilizador válidas para cada jogador e, em seguida, atualizando o estado do jogo (gerido pela classe ConnectState). No final de cada jogada, o estado do jogo é verificado para ver se existe um vencedor e, se o estado devolver o vencedor como o jogador que acabou de jogar, indica que o jogador atual ganhou; caso contrário, indica que o jogo terminou empatado.

In [ ]:
def playPvP():
    state = ConnectState()

    while not state.game_over():
        print("Current state:")
        state.print_board()

        # --- Player 1 Move ---
        while True:
            p1_input = input("Player 1 ('X') enter a move: ")
            if not p1_input.strip():
                print("You need to enter a move.")
                continue
            try:
                p1Move = int(p1_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue

            if p1Move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break

        state.move(p1Move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        # --- Player 2 Move ---
        while True:
            p2_input = input("Player 2 ('O') enter a move: ")
            if not p2_input.strip():
                print("You need to enter a move.")
                continue
            try:
                p2Move = int(p2_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue

            if p2Move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break

        state.move(p2Move)
        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break

### 2. Humano x Computador (MCTS)

descrição...

In [ ]:
def playPvC1():
    state = ConnectState()
    mcts = MCTS(state)
    changed_To_Attack = False # se o valor de C foi mudado ou não
    num_Plays = 0

    while not state.game_over():
        print("Current state:")
        state.print_board()

        while True:
            user_input = input("Enter a move: ")
            # Check if input is blank or not a number
            if not user_input.strip():
                print("You need to enter a move.")
                continue
            try:
                user_move = int(user_input) - 1
            except ValueError:
                print("That's not a valid number. Try again.")
                continue
            if user_move not in state.get_legal_moves():
                print("Illegal move")
                continue
            break  # valid input and legal move

        state.move(user_move)
        mcts.move(user_move)

        state.print_board()

        if state.game_over():
            result = state.get_result()
            if result == GameMeta.OUTCOMES['one']:
                print("Player one ('X') won!")
            else:
                print("Draw!")
            break

        print("Thinking...")

        mcts.search(10)
        num_rollouts, run_time = mcts.statistics()
        print(f"Statistics: {num_rollouts} rollouts in {run_time:.2f} seconds")
        move = mcts.best_move()
        state.move(move)
        mcts.move(move)

        
        if state.game_over():
            state.print_board()
            result = state.get_result()
            if result == GameMeta.OUTCOMES['two']:
                print("Player two ('O') won!")
            else:
                print("Draw!")
            break

        if not changed_To_Attack:
            num_Plays += 2
            if num_Plays > 17:
                mcts.change_c_value()
                changed_To_Attack = True